In [3]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse
import math
from collections import Counter
import sys

In [4]:
sys.path.append('./feature_detection')
from url_entropy import shannon_entropy, relative_entropy, probabilities

In [7]:
# Cargar el dataset original
df = pd.read_csv('../dataset_pishing.csv')

print(f"Dataset original: {df.shape}")
print(f"Columnas: {df.columns.tolist()}")
print(df.head())

Dataset original: (11430, 2)
Columnas: ['url', 'status']
                                                 url      status
0              http://www.crestonwood.com/router.php  legitimate
1  http://shadetreetechnology.com/V4/validation/a...    phishing
2  https://support-appleld.com.secureupdate.duila...    phishing
3                                 http://rgipt.ac.in  legitimate
4  http://www.iracing.com/tracks/gateway-motorspo...  legitimate


## Funciones de extracción

In [8]:
# 1. Longitud total de la URL
def url_length(url):
    """Calcula la longitud total de la URL"""
    return len(url)

In [9]:
# 2. Longitud del dominio
def domain_length(url):
    """Calcula la longitud del nombre de dominio"""
    try:
        parsed = urlparse(url)
        domain = parsed.netloc
        return len(domain)
    except:
        return 0

In [10]:
# 3. Longitud del path
def path_length(url):
    """Calcula la longitud del path"""
    try:
        parsed = urlparse(url)
        return len(parsed.path)
    except:
        return 0


In [11]:
# 4. Número de puntos en la URL
def count_dots(url):
    """Cuenta el número de puntos en la URL"""
    return url.count('.')

In [12]:
# 5. Número de guiones en la URL
def count_hyphens(url):
    """Cuenta el número de guiones"""
    return url.count('-')

In [13]:
# 6. Número de subdominios
def count_subdomains(url):
    """Cuenta el número de subdominios"""
    try:
        parsed = urlparse(url)
        domain = parsed.netloc
        # Eliminar puerto si existe
        domain = domain.split(':')[0]
        # Contar puntos en el dominio (subdominios = puntos - 1 típicamente)
        dots = domain.count('.')
        return max(0, dots - 1)  # Restar 1 para el TLD
    except:
        return 0

In [14]:
# 7. Uso de IP en lugar de dominio (binario)
def has_ip_address(url):
    """Detecta si se usa una dirección IP en lugar de dominio"""
    try:
        parsed = urlparse(url)
        domain = parsed.netloc.split(':')[0]
        # Patrón para IPv4
        ipv4_pattern = r'^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$'
        return 1 if re.match(ipv4_pattern, domain) else 0
    except:
        return 0

In [15]:
# 8. Presencia de @ en la URL
def has_at_symbol(url):
    """Detecta presencia del símbolo @"""
    return 1 if '@' in url else 0

In [16]:
# 9. Presencia de // en el path (redirección)
def has_double_slash_redirect(url):
    """Detecta // en el path que puede indicar redirección"""
    try:
        parsed = urlparse(url)
        path = parsed.path
        return 1 if '//' in path else 0
    except:
        return 0

In [17]:
# 10. Uso de HTTPS
def is_https(url):
    """Verifica si usa protocolo HTTPS"""
    return 1 if url.startswith('https://') else 0

In [18]:
# 11. Número de parámetros en query string
def count_query_params(url):
    """Cuenta el número de parámetros en la query string"""
    try:
        parsed = urlparse(url)
        if parsed.query:
            return len(parsed.query.split('&'))
        return 0
    except:
        return 0

In [19]:
# 12. Presencia de palabras sensibles
def has_sensitive_words(url):
    """Detecta palabras sensibles comunes en phishing"""
    sensitive_words = ['login', 'signin', 'account', 'update', 'verify', 
                      'secure', 'banking', 'confirm', 'password', 'credential']
    url_lower = url.lower()
    return sum(1 for word in sensitive_words if word in url_lower)

In [ ]:
# 13. Número de caracteres especiales
def count_special_chars(url):
    """Cuenta caracteres especiales (no alfanuméricos)"""
    return sum(1 for c in url if not c.isalnum())

In [21]:
# 14. Ratio de dígitos en la URL
def digit_ratio(url):
    """Calcula el ratio de dígitos respecto al total"""
    if len(url) == 0:
        return 0
    digit_count = sum(1 for c in url if c.isdigit())
    return digit_count / len(url)

In [22]:
# 15. Número de vocales en el dominio
def count_vowels_in_domain(url):
    """Cuenta vocales en el nombre de dominio"""
    try:
        parsed = urlparse(url)
        domain = parsed.netloc.split(':')[0]
        vowels = 'aeiouAEIOU'
        return sum(1 for c in domain if c in vowels)
    except:
        return 0

In [23]:
# 16. Presencia de prefijo/sufijo con guión en dominio
def has_prefix_suffix(url):
    """Detecta guiones en el dominio (técnica común de phishing)"""
    try:
        parsed = urlparse(url)
        domain = parsed.netloc.split(':')[0]
        # Separar por puntos para obtener solo el SLD
        parts = domain.split('.')
        if len(parts) >= 2:
            sld = parts[-2]  # Second Level Domain
            return 1 if '-' in sld else 0
        return 0
    except:
        return 0

In [24]:
# 17. Longitud promedio de palabras en el path
def avg_word_length_path(url):
    """Calcula la longitud promedio de palabras en el path"""
    try:
        parsed = urlparse(url)
        path = parsed.path
        # Separar por caracteres no alfanuméricos
        words = re.findall(r'[a-zA-Z]+', path)
        if len(words) == 0:
            return 0
        return sum(len(w) for w in words) / len(words)
    except:
        return 0

In [25]:
# 18. Número de símbolos & (parámetros múltiples)
def count_ampersands(url):
    """Cuenta el número de ampersands"""
    return url.count('&')

In [26]:
# 19. Número de símbolos = (parámetros)
def count_equals(url):
    """Cuenta el número de signos igual"""
    return url.count('=')

In [27]:
# 20. Detección de URL acortada (shortening service)
def is_shortened_url(url):
    """Detecta servicios de acortamiento de URLs"""
    shortening_services = ['bit.ly', 'goo.gl', 'tinyurl.com', 'ow.ly', 
                          't.co', 'is.gd', 'buff.ly', 'adf.ly']
    url_lower = url.lower()
    return 1 if any(service in url_lower for service in shortening_services) else 0

In [28]:
# 21. Entropía de Shannon (importada)
def shannon_entropy_feature(url):
    """Calcula la entropía de Shannon de la URL"""
    return shannon_entropy(url)

In [29]:
# 22. Entropía relativa (importada)
def relative_entropy_feature(url):
    """Calcula la entropía relativa de la URL"""
    return relative_entropy(url, probabilities)

In [30]:
# 23. Número de barras en la URL
def count_slashes(url):
    """Cuenta el número de barras (/)"""
    return url.count('/')

## Aplicar funciones a dataset

In [31]:
# Aplicar todas las funciones
df['url_length'] = df['url'].apply(url_length)
df['domain_length'] = df['url'].apply(domain_length)
df['path_length'] = df['url'].apply(path_length)
df['count_dots'] = df['url'].apply(count_dots)
df['count_hyphens'] = df['url'].apply(count_hyphens)
df['count_subdomains'] = df['url'].apply(count_subdomains)
df['has_ip'] = df['url'].apply(has_ip_address)
df['has_at'] = df['url'].apply(has_at_symbol)
df['has_double_slash'] = df['url'].apply(has_double_slash_redirect)
df['is_https'] = df['url'].apply(is_https)
df['num_params'] = df['url'].apply(count_query_params)
df['sensitive_words'] = df['url'].apply(has_sensitive_words)
df['special_chars'] = df['url'].apply(count_special_chars)
df['digit_ratio'] = df['url'].apply(digit_ratio)
df['vowels_in_domain'] = df['url'].apply(count_vowels_in_domain)
df['has_prefix_suffix'] = df['url'].apply(has_prefix_suffix)
df['avg_word_len_path'] = df['url'].apply(avg_word_length_path)
df['count_ampersands'] = df['url'].apply(count_ampersands)
df['count_equals'] = df['url'].apply(count_equals)
df['is_shortened'] = df['url'].apply(is_shortened_url)
df['shannon_entropy'] = df['url'].apply(shannon_entropy_feature)
df['relative_entropy'] = df['url'].apply(relative_entropy_feature)
df['count_slashes'] = df['url'].apply(count_slashes)

In [32]:
print(f"\nDataset expandido: {df.shape}")
print(f"Nuevas características añadidas: {df.shape[1] - 2}")  # -2 por url y status


Dataset expandido: (11430, 25)
Nuevas características añadidas: 23


In [33]:
# Mostrar estadísticas descriptivas de las nuevas características
print("\nEstadísticas descriptivas de las nuevas características:")
print(df.describe())


Estadísticas descriptivas de las nuevas características:
         url_length  domain_length   path_length    count_dots  count_hyphens  \
count  11430.000000   11430.000000  11430.000000  11430.000000   11430.000000   
mean      61.120035      21.100175     23.146107      2.480665       0.997550   
std       55.292470      10.778330     27.738075      1.369685       2.087087   
min       12.000000       4.000000      0.000000      1.000000       0.000000   
25%       33.000000      15.000000      1.000000      2.000000       0.000000   
50%       47.000000      19.000000     17.000000      2.000000       0.000000   
75%       71.000000      24.000000     33.000000      3.000000       1.000000   
max     1641.000000     214.000000    602.000000     24.000000      43.000000   

       count_subdomains        has_ip        has_at  has_double_slash  \
count      11430.000000  11430.000000  11430.000000      11430.000000   
mean           1.052493      0.008486      0.021435          0.002

## Guardar resultado

In [35]:
output_filename = '../dataset_phishing_extended.csv'
df.to_csv(output_filename, index=False)
print(f"\nDataset guardado como: {output_filename}")


Dataset guardado como: ../dataset_phishing_extended.csv
